## Greenhouse emission data preparation

In [20]:
# Import libraries
import pandas as pd 
import project_libs as libs
import seaborn as sns 

In [21]:
#Load Emission data
total_green_house_emission = libs.read_csv_to_dataframe('data/total-ghg-emissions.csv')
total_green_house_emission.rename(columns={'Entity':'Country'}, inplace=True)

shape (37195, 4)
----------------------------------------------------------------------------------------------------
List of columns
['Entity', 'Code', 'Year', 'Annual greenhouse gas emissions in CO₂ equivalents']
----------------------------------------------------------------------------------------------------
Data info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37195 entries, 0 to 37194
Data columns (total 4 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   Entity                                              37195 non-null  object 
 1   Code                                                34081 non-null  object 
 2   Year                                                37195 non-null  int64  
 3   Annual greenhouse gas emissions in CO₂ equivalents  37195 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 1.1+ MB
None
-------------

In [22]:
#Load cap data
counties_with_cap = libs.read_csv_to_dataframe('data/countries with cap and trade.csv')

shape (42, 3)
----------------------------------------------------------------------------------------------------
List of columns
['Country', 'Year', 'Cap_and_trade']
----------------------------------------------------------------------------------------------------
Data info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Country        42 non-null     object
 1   Year           42 non-null     int64 
 2   Cap_and_trade  42 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ KB
None
----------------------------------------------------------------------------------------------------


In [23]:
display(total_green_house_emission.head())
print('-'* 100)
display(counties_with_cap.head())

,Country,Code,Year,Annual greenhouse gas emissions in CO₂ equivalents
0,Afghanistan,AFG,1850,7338819.0
1,Afghanistan,AFG,1851,7403982.5
2,Afghanistan,AFG,1852,7464157.0
3,Afghanistan,AFG,1853,7522431.5
4,Afghanistan,AFG,1854,7579209.5


----------------------------------------------------------------------------------------------------


,Country,Year,Cap_and_trade
0,Austria,2005,1
1,Belgium,2005,1
2,Bulgaria,2005,1
3,Croatia,2005,1
4,Cyprus,2005,1


In [24]:
# Join Emission and Carbon cap data to get cap label
emission_cap_data = pd.merge(total_green_house_emission, counties_with_cap, on=['Country', 'Year'], how='left')
#Set cap and trade na values to zero (countries with no cap)
emission_cap_data['Cap_and_trade'].fillna(0,inplace=True)

#Check countries that will not join
list1, list2 = counties_with_cap['Country'].unique().tolist(), total_green_house_emission['Country'].unique().tolist()

# Check if the lists are not identical
print([item for item in list1 if item not in list2])

['Czech Republic']


C:\Users\hafto\AppData\Local\Temp\ipykernel_19684\1345894768.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  emission_cap_data['Cap_and_trade'].fillna(0,inplace=True)


In [25]:
#view cap counts
emission_cap_data.head()

,Country,Code,Year,Annual greenhouse gas emissions in CO₂ equivalents,Cap_and_trade
0,Afghanistan,AFG,1850,7338819.0,0.0
1,Afghanistan,AFG,1851,7403982.5,0.0
2,Afghanistan,AFG,1852,7464157.0,0.0
3,Afghanistan,AFG,1853,7522431.5,0.0
4,Afghanistan,AFG,1854,7579209.5,0.0


## Load and clean emission target data by country and year

In [26]:
#laod target data
target = libs.read_csv_to_dataframe('data/target.csv')

shape (22764, 73)
----------------------------------------------------------------------------------------------------
List of columns
['Country', 'ISO2', 'ISO3', 'Indicator', 'Unit', 'Source', 'CTS Code', 'CTS Name', 'CTS Full Descriptor', 'Industry', 'Gas Type', 'Scale', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024', '2025', '2026', '2027', '2028', '2029', '2030']
----------------------------------------------------------------------------------------------------
Data info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22764 entries, 0 to 22763
Data columns (total 73 columns):
 #   Column       

In [27]:
#Melt and aggrigate target emission values
target_column_list = target.select_dtypes(include = 'number').columns.to_list()
target_column_list.append('Country')

target_emission_data = target[target_column_list]
target_emission  = target_emission_data.melt(id_vars=['Country'], var_name='Year', value_name='emission_target').groupby(['Country', 'Year']).agg({'emission_target': 'sum'}).reset_index()
target_emission ['Year']= target_emission['Year'].astype('int64')
#Display head
target_emission.head()

,Country,Year,emission_target
0,Advanced Economies,1970,117279.931241
1,Advanced Economies,1971,116247.111022
2,Advanced Economies,1972,120931.530411
3,Advanced Economies,1973,126689.687342
4,Advanced Economies,1974,123634.675667


In [28]:
#Combine emission and target data by country and year
emission_cap_target_data = pd.merge(emission_cap_data, target_emission, on=['Country', 'Year'], how='left')

#display tail
emission_cap_target_data.tail()

,Country,Code,Year,Annual greenhouse gas emissions in CO₂ equivalents,Cap_and_trade,emission_target
37198,Zimbabwe,ZWE,2018,37847984.0,0.0,602.030661
37199,Zimbabwe,ZWE,2019,36351330.0,0.0,590.823283
37200,Zimbabwe,ZWE,2020,33081062.0,0.0,570.481972
37201,Zimbabwe,ZWE,2021,33889200.0,0.0,580.824175
37202,Zimbabwe,ZWE,2022,34333068.0,0.0,744.094865


## Read surface temprature by year and country

In [29]:
#laod targetco2 concentration data
surface_temp = libs.read_csv_to_dataframe('data/23_Annual_Surface_Temperature_Change.csv')

shape (225, 71)
----------------------------------------------------------------------------------------------------
List of columns
['Country', 'ISO2', 'ISO3', 'Indicator', 'Unit', 'Source', 'CTS Code', 'CTS Name', 'CTS Full Descriptor', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022']
----------------------------------------------------------------------------------------------------
Data info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225 entries, 0 to 224
Data columns (total 71 columns):
 #   Column               Non-Null Count  Dtype  

In [30]:
#display head
surface_temp.head()

,Country,ISO2,ISO3,Indicator,Unit,Source,CTS Code,CTS Name,CTS Full Descriptor,1961,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,"Afghanistan, Islamic Rep. of",AF,AFG,Temperature change with respect to a baseline ...,Degree Celsius,Food and Agriculture Organization of the Unite...,ECCS,Surface Temperature Change,"Environment, Climate Change, Climate Indicator...",-0.113,...,1.281,0.456,1.093,1.555,1.540,1.544,0.910,0.498,1.327,2.012
1,Albania,AL,ALB,Temperature change with respect to a baseline ...,Degree Celsius,Food and Agriculture Organization of the Unite...,ECCS,Surface Temperature Change,"Environment, Climate Change, Climate Indicator...",0.627,...,1.333,1.198,1.569,1.464,1.121,2.028,1.675,1.498,1.536,1.518
2,Algeria,DZ,DZA,Temperature change with respect to a baseline ...,Degree Celsius,Food and Agriculture Organization of the Unite...,ECCS,Surface Temperature Change,"Environment, Climate Change, Climate Indicator...",0.164,...,1.192,1.690,1.121,1.757,1.512,1.210,1.115,1.926,2.330,1.688
3,American Samoa,AS,ASM,Temperature change with respect to a baseline ...,Degree Celsius,Food and Agriculture Organization of the Unite...,ECCS,Surface Temperature Change,"Environment, Climate Change, Climate Indicator...",0.079,...,1.257,1.170,1.009,1.539,1.435,1.189,1.539,1.430,1.268,1.256
4,"Andorra, Principality of",AD,AND,Temperature change with respect to a baseline ...,Degree Celsius,Food and Agriculture Organization of the Unite...,ECCS,Surface Temperature Change,"Environment, Climate Change, Climate Indicator...",0.736,...,0.831,1.946,1.690,1.990,1.925,1.919,1.964,2.562,1.533,3.243


In [31]:
#Melt and aggrigate temprature data
col_list = surface_temp.select_dtypes(include = 'number').columns.tolist()
col_list.append('Country')

yearly_surface_tem_change = surface_temp[col_list]

yearly_surface_tem_change = yearly_surface_tem_change.melt(id_vars=['Country'], var_name='Year', value_name='temp_change').groupby(['Country', 'Year']).agg({'temp_change': 'mean'}).reset_index()

#display head
yearly_surface_tem_change.head()

,Country,Year,temp_change
0,"Afghanistan, Islamic Rep. of",1961,-0.113
1,"Afghanistan, Islamic Rep. of",1962,-0.164
2,"Afghanistan, Islamic Rep. of",1963,0.847
3,"Afghanistan, Islamic Rep. of",1964,-0.764
4,"Afghanistan, Islamic Rep. of",1965,-0.244


In [32]:
#Combine temprature change data with emission data
yearly_surface_tem_change ['Year']= target_emission['Year'].astype('int64')

emission_cap_target_data_temp = pd.merge(emission_cap_target_data, yearly_surface_tem_change, on=['Country', 'Year'], how='left')

emission_cap_target_data_temp.tail()

,Country,Code,Year,Annual greenhouse gas emissions in CO₂ equivalents,Cap_and_trade,emission_target,temp_change
37315,Zimbabwe,ZWE,2018,37847984.0,0.0,602.030661,0.093
37316,Zimbabwe,ZWE,2019,36351330.0,0.0,590.823283,0.215
37317,Zimbabwe,ZWE,2020,33081062.0,0.0,570.481972,0.421
37318,Zimbabwe,ZWE,2021,33889200.0,0.0,580.824175,-0.052
37319,Zimbabwe,ZWE,2022,34333068.0,0.0,744.094865,-0.397


## Load and clean renewable energy data by year and country 

In [33]:
#laod targetco2 concentration data
renewable_energy = libs.read_csv_to_dataframe('data/10_Renewable_Energy.csv')

renewable_list = renewable_energy.select_dtypes(include = 'number').columns.tolist()
renewable_list.append('Country')

shape (2617, 34)
----------------------------------------------------------------------------------------------------
List of columns
['Country', 'ISO2', 'ISO3', 'Indicator', 'Technology', 'Energy Type', 'Unit', 'Source', 'CTS Name', 'CTS Code', 'CTS Full Descriptor', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022']
----------------------------------------------------------------------------------------------------
Data info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2617 entries, 0 to 2616
Data columns (total 34 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Country              2617 non-null   object 
 1   ISO2                 2139 non-null   object 
 2   ISO3                 2617 non-null   object 
 3   Indicator            2617 non-null   object 
 4   Technology           261

In [34]:
#Filter renewable energy data to only country and year and value
renewable_list.remove('Source')

#select only required columns
renewable_energy_data  = renewable_energy[renewable_list]

#Melt and aggrigate
renewable_energy_data = renewable_energy_data.melt(id_vars=['Country'], var_name='Year', value_name='renewable_energy').groupby(['Country', 'Year']).agg({'renewable_energy': 'mean'}).reset_index()

#Display head
renewable_energy_data.head()

,Country,Year,renewable_energy
0,Advanced Economies,2000,564735.72180
1,Advanced Economies,2001,558368.03965
2,Advanced Economies,2002,576313.75415
3,Advanced Economies,2003,582884.29605
4,Advanced Economies,2004,596069.97165


In [35]:
#Combine renewable energy data to master data
renewable_energy_data ['Year']= renewable_energy_data['Year'].astype('int64')

master_emission_data = pd.merge(emission_cap_target_data_temp, renewable_energy_data, on=['Country', 'Year'], how='left')

#Remove Code
master_emission_data.drop(columns='Code',inplace=True,axis=1)

#display tail

master_emission_data.tail()

,Country,Year,Annual greenhouse gas emissions in CO₂ equivalents,Cap_and_trade,emission_target,temp_change,renewable_energy
37315,Zimbabwe,2018,37847984.0,0.0,602.030661,0.093,1494.560625
37316,Zimbabwe,2019,36351330.0,0.0,590.823283,0.215,1478.418750
37317,Zimbabwe,2020,33081062.0,0.0,570.481972,0.421,1116.044500
37318,Zimbabwe,2021,33889200.0,0.0,580.824175,-0.052,1391.685875
37319,Zimbabwe,2022,34333068.0,0.0,744.094865,-0.397,618.003500


In [36]:
from sklearn.impute import SimpleImputer
import pandas as pd

# Assuming df is your DataFrame with missing values
# Create an instance of SimpleImputer
imputer = SimpleImputer(strategy='most_frequent')  # Other strategies: median, most_frequent, constant

# Fit and transform the data
master_data = pd.DataFrame(imputer.fit_transform(master_emission_data), columns=master_emission_data.columns)

In [37]:
master_data.to_csv('data/master_emission_data.csv', index= False)

In [40]:
master_data.isna().sum()

Country                                               0
Year                                                  0
Annual greenhouse gas emissions in CO₂ equivalents    0
Cap_and_trade                                         0
emission_target                                       0
temp_change                                           0
renewable_energy                                      0
dtype: int64